# Web scraping

_Data Structures and Algorithms_

_Imperial College Business School_


---
In this tutorial we'll learn the basics of web scraping with Python using the `BeautifulSoup` and `requests` libraries.

---


## Submission

This tutorial is optional and meant to be open-ended, and you don't need to submit anything. 

## Accessing the web with Python

Most websites are written in HTML, which stands for HyperText Markup Language. These pages have the extension `.html` in our browser. An HTML file is a type of a text file, which includes the content of a web page, but also instructions for the browser on how to display the content, for example in paragraphs, tables, bold font, links, and so on. 

When we access a web page with Python, we'll replicate the steps that our web browser does when we visit a page. We contact a website and download the HTML text. We'll then take advantage of the HTML structure (e.g. identifiers for links) to extract the information we're interested in. 

The process of programmatically gathering data from websites is often called _scraping_ the web. If we need to download data from many pages, or repeatedly, this automating web access using Python libraries can be a very powerful. However, as we will see, parsing information from HTML files can be fragile and cumbersome. For web sites that provide direct ways to access their data in more structured form through, access through their APIs is typically a much better choice.  

We'll first go through the basics of HTML and Python's web access libraries. We'll then look at the important problem of finding a good breakfast in the UK. 

### HTML basics

HTML files are just text files, but how is HTML different from regular text then? Here is an example of three paragraphs in plain text and in HTML.

Text:
```text
Hello, world!

Hello again.

An example page: http://www.example.com/
```

Same in HTML:
```html
<p>Hello, world!</p>

<p>Hello again.</p>

<p><a href="http://www.example.com/">An example page</a></p>
```

In addition to the text content, the HTML version contains **tags**. These are contained within angle brackets: the tag `<p>` is a starting tag, and `</p>` is the corresponding ending tag. In between the tags, there is the content of the _(p)aragraph_ of text. 

These tags can be nested.
```html
<p>Hello, <strong>world!</strong></p>

<p>Hello again.</p>
```
Here `<p>` and `<strong>` are _parent_ and _child_ tags, and the two `<p>` tags are _sibling_ tags.

Tags can further have **attributes**. For example, the hyperlink tag `<a>` (where "a" is for "anchor") specifies the target URL as the `href` attribute in the opening tag:
```html
<p><a href="http://www.example.com/">An example page</a></p>

<p id="myid">Hey there</p>
```
Web sites use custom attributes to make content look fancy with CSS (cascading style sheets): for example, they can determine how all elements with the `id` attribute `myid` look (the `class` attribute is also common for CSS styling). The syntax of these is the name of the attribute, followed by an equals sign, followed by a quoted value. Attributes are always placed in the opening tag.


### Parsing HTML with BeautifulSoup

We have seen how we can loop through text, split it into words, and so on. We can similarly parse a HTML file to extract information we're looking for. Luckily, there are great libraries that automate much of this process for us. 

We'll use the BeautifulSoup 4 library. Similarly to a web browser, Beautiful Soup takes advantage of the tag structure of the web page, not to display it, but to find the information that we'd like to store. For example, we may want to find all the links in a web page, so we would look for all the `<a href...>` tags. 

Let's import the library and parse some HTML text. The code below creates a `BeautifulSoup` object that the Beautiful Soup library works with.

In [5]:
from bs4 import BeautifulSoup
html = '<p>Hello, world!</p> <p>Hello again.</p>'
soup = BeautifulSoup(html, 'lxml') # two arguments: text and 'lxml', which is the html parser BS uses
print(soup.text)
print(type(soup))

Hello, world! Hello again.
<class 'bs4.BeautifulSoup'>


We can find the text contained in the data through the `text` attribute. What if we want to find all tags, here the paragraph tags? We can use the `find_all` method from the library.

In [9]:
paragraphs = soup.find_all('p') # find all <p> tags
print(paragraphs)
print(type(paragraphs)) # this is an augmented list-type object

[<p>Hello, world!</p>, <p>Hello again.</p>]
<class 'bs4.element.ResultSet'>


The result is a list-type object which contains the tags. We can access it just like a list, and get the text of the contents.

In [10]:
first = paragraphs[0]
print(first.name)
print(first.text)
print(type(first))

p
Hello, world!
<class 'bs4.element.Tag'>


In practice, finding the right tags often involves some detective work in deciding which tags we're looking for. Suppose that we're interested in scraping information from the website [www.example.com](http://www.example.com). The website is simple, with just one link on the page. If we want to see how the underlying HTML looks like, we can access this information in our browser's *developer tools*; in most browsers the shortcut for viewing the source HTML of a page is `Ctrl + U`. In the example domain, a snippet of the HTML containing the displayed text looks like this:
```html
<div>
    <h1>Example Domain</h1>
    <p>This domain is established to be used for illustrative examples in documents. You may use this
    domain in examples without prior coordination or asking for permission.</p>
    <p><a href="http://www.iana.org/domains/example">More information...</a></p>
</div>
```
Viewing the HTML like this is useful for deciding which tags we may want to search for.

### Browsing with Python

We'll use the `requests` library to access web pages. It downloads the web page, here the one at [www.example.com](http://www.example.com).

In [15]:
import requests
r = requests.get('http://www.example.com')

The result, stored into the variable `r`, is an object containing the raw html string. We can access through the `text` attribute.

In [ ]:
print(r.ok) # was the download attempt successful
r.text # raw HTML

Now we can use Beautiful Soup to find all the links in the page.

In [14]:
soup = BeautifulSoup(r.text, 'lxml')
links = soup.find_all('a')
print(links)

[<a href="http://www.iana.org/domains/example">More information...</a>]


Now, we'd like to access the accual link, that is, the `href` attribute for the link. We can get to this with the `attrs` attribute of each of the found links. This is a dictionary of attribute-value pairs, and we can get the relevant one by accessing the `href` key.

In [20]:
first_link = links[0]
print(first_link.text)
print(first_link.attrs) # dictionary of attributes
print(first_link.attrs['href'])

More information...
{'href': 'http://www.iana.org/domains/example'}
http://www.iana.org/domains/example


Next, let's look at a more realistic example of finding information in a web page.

### Example: Best UK breakfasts from the Guardian

[This article from the Guardian](https://www.theguardian.com/lifeandstyle/2017/jan/15/50-best-breakfasts-uk) purports to list the 50 best breakfasts in the UK. The list is interesting, but it is in an unstructured form. If we wanted to remember these whenever we're visiting a different city, we'd probably like to store them in a file on our computer or online for easy access. We'll use Python to make this transformation.

The web page lists the restaurants, giving links, locations, and a short description of each. Let's look at how the underlying HTML file looks like. When viewing a web page in a browser, we can typically do this with the shortcut `Ctrl + U`. This will open the HTML source in a new tab.

The file is huge, with 3000 lines of text. We won't go into details of the HTML here, but instead look for the part in the file where the restaurants are presented. The first restaurant is in "Lewannick", so let's look for that in the source with `Ctrl + F`. We see the following result.

```html
<h2><strong><a href="http://www.coombesheadfarm.co.uk/" data-link-name="in body link" class="u-underline">Coombeshead Farm</a></strong><strong>, Lewannick, Cornwall</strong><br></h2>
<p>It might seem a bit bonkers to include an upmarket B&amp;B (from £90 a night) that doesn’t offer breakfast to non-residents (for the time being), but Coombeshead earns its place here because it does possibly the best breakfast in the UK, and certainly this writer’s favourite of 2016 anywhere.</p>
```

Compare this to the website as it is displayed in your browser and try to link the different parts together.


Let's try to understand the first restaurant details from the HTML code. Above, the text ", Lewannick, Cornwall" is packaged in between the HTML _tags_ `<strong>` and `</strong>`. The web browser uses HTML tags to display contents in a specific way, here in **bold** font. The tag `<strong>` is the opening tag which starts the emphasized text, and `</strong>` with a slash is the closing tag which ends the emphasis.


Similarly, the text "Coombeshead Farm" is packaged in between the tags `<a>` and `</a>`:
```html
<a href="http://www.coombesheadfarm.co.uk/" data-link-name="in body link" class="u-underline">Coombeshead Farm</a>
```
As we've seen, this is the HTML way of specifying a link to another page. The tag `<a href="http://www.coombesheadfarm.co.uk/">` is the opening tag which creates the link to the restaurant's website. The tag `</a>` closes the tag: in between these, the text "Coombeshead Farm" is what gets displayed on the web page as the link text. Here, the tag has three _attributes_: `href` specifying the link address, and `data-link-name="in body link"` and `class="u-underline"`, which the browser uses to determine how the link is shown.

There are many different HTML tags, which the web browser uses to display contents in a specific way. We don't need to know their meanings for scraping purposes; we will simply use them to pick the data we'd like from the page.

### Getting the data

Let's download the HTML text using the `requests` library. The code below fetches the page, and stores the result in `r`, which is a "response" object that the library creates. The object has a `text` attribute, which contains the HTML of the page.

In [16]:
import requests
r = requests.get("https://www.theguardian.com/lifeandstyle/2017/jan/15/50-best-breakfasts-uk")
r.text[:500]

'\n<!DOCTYPE html>\n<html id="js-context" class="js-off is-not-modern id--signed-out" lang="en" data-page-path="/lifeandstyle/2017/jan/15/50-best-breakfasts-uk">\n<head>\n<!--\n     __        __                      _     _      _\n     \\ \\      / /__    __ _ _ __ ___  | |__ (_)_ __(_)_ __   __ _\n      \\ \\ /\\ / / _ \\  / _` | \'__/ _ \\ | \'_ \\| | \'__| | \'_ \\ / _` |\n       \\ V  V /  __/ | (_| | | |  __/ | | | | | |  | | | | | (_| |\n        \\_/\\_/ \\___|  \\__,_|_|  \\___| |_| |_|_|_|  |_|_| |_|\\__, |\n        '

Let's use Beautiful Soup to parse the text. The code below creates a `soup` object that the Beautiful Soup library works with.

In [24]:
from bs4 import BeautifulSoup  
soup = BeautifulSoup(r.text, 'lxml')  

### Collecting restaurants

We'd like to get all 50 restaurants' names, locations, and websites. Let's take another look at the HTML source.

```html
<h2><strong><a href="http://www.coombesheadfarm.co.uk/" data-link-name="in body link" class="u-underline">Coombeshead Farm</a></strong><strong>, Lewannick, Cornwall</strong><br></h2>
<p>It might seem a bit bonkers to include an upmarket B&amp;B (from £90 a night) that doesn’t offer breakfast to non-residents (for the time being), but Coombeshead earns its place here because it does possibly the best breakfast in the UK, and certainly this writer’s favourite of 2016 anywhere.</p>
```

It looks like the structure of the data we'd like is roughly the following.
```html
<h2><strong><a href="WEBPAGE LINK" data-link-name="in body link" class="u-underline">RESTAURANT NAME</a></strong><strong>, LOCATION</strong><br></h2>
```
There's an outer `<h2>` tag which specifies a header, a `<strong>` tag and an `<a>` tag fpr the link and the name, and another `<strong>` tag for the location.

We'd like to pick the three capitalized parts for all restaurants. We can search for tags in Beautiful Soup as follows.

In [236]:
results = soup.find_all('h2') # find all h2 tags
print(results[0:2])

[<h2><strong><a class="u-underline" data-link-name="in body link" href="http://www.coombesheadfarm.co.uk/">Coombeshead Farm</a></strong><strong>, Lewannick, Cornwall</strong><br/></h2>, <h2><a class="u-underline" data-link-name="in body link" href="http://cafealfresco.co.uk/page/eat"><strong>Café Alf Resco</strong></a><strong>, Dartmouth, Devon</strong></h2>]


Let's see how we can extract the information we want from each restaurant. We can take the first result we found, and apply Beautiful Soup's find method to it again to find (the first) link. The link will have the _attribute_ text and contain the actual link as `link['href']`.

In [250]:
# Find link in first result
link = results[0].find('a') # find link
name = link.text # get text
url = link['href'] # get link address
print(name, url)

# Find the location
location = results[0].find_all('strong')[1].text
location = location.lstrip(' ,') # get rid of leading comma and space
print(location)

Coombeshead Farm http://www.coombesheadfarm.co.uk/
Lewannick, Cornwall


### Scraping is messy

Now try to write a loop that repeats this for all the restaurants. Store the results in a list containing dictionary entries as follows:
```python
[{'Restaurant':name, 'Web page':url, 'Location':location}]
```

At some point you will run into trouble. This is because we have not just picked all restaurants, but other things in the article that also have the `<h2>` tag. Some of the results will thus not contain the information we want, and we need to prune them  further. Try to work your loop around this, and other issues you'll run into, to pick the 50 restaurants.

In [234]:
# Your code here
result_list = []


Once you have your results, you can use the following code to write the dictionary into a csv file with the `csv` library.

In [ ]:
import csv

# File column names
csv_columns = ['Location','Restaurant','Web page']

with open('best_breakfast_guardian.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader() # header row
    for rest in result_list: # rest of rows
        writer.writerow(rest)

What if we also wanted to store the description of each restaurant? One approach to do so is to use the hierarchical structure of the file's HTML tags with Beautiful Soup. We say that the tags at the same level in the hierarchy as our current tag are _siblings_, while those above (nesting the current tag) are _parents_ and those below (inside the current tag) are _children_.  

Inspecting the HTML, we see that the tags between two `<h2>` tags are `<p>` tags for paragraphs. Starting from a `<h2>` tag, we can loop through the paragraphs with the `next_sibling` attribute until we run into the next `<h2>` tag.

In [248]:
current = results[0]

text = ''
# Loop until find next restaurant, ie h2 tag
while current.next_sibling.name != 'h2':
    print(current.next_sibling) # Either <p> or a new line
    current = current.next_sibling
    if current.name is not None: # Only get the paragraph text
        text += current.text + ' ' # Add a space between paragraphs
print('Here\'s the full text:\n')
print(text)



<p>It might seem a bit bonkers to include an upmarket B&amp;B (from £90 a night) that doesn’t offer breakfast to non-residents (for the time being), but Coombeshead earns its place here because it does possibly the best breakfast in the UK, and certainly this writer’s favourite of 2016 anywhere.</p>


<p>The love child of two celebrated chefs, <a class="u-underline" data-link-name="in body link" href="http://aprilbloomfield.com/">April Bloomfield</a> (of The Spotted Pig and The Breslin in New York) and Tom Adams (of <a class="u-underline" data-link-name="in body link" href="http://www.pittcue.co.uk/">Pittcue in London</a>), this idyllic Cornish farmhouse, set in 60-plus acres of picture-postcard meadow and woodland, serves up an equally idyllic breakfast. Adams and co go the extra mile over the most important meal of the day: everything is homegrown, made on site or sourced as locally as possible.</p>


<p>From the granola, bircher muesli, honey and on-trend kombucha (for those who l

## Summary

As you've seen with the example, web scraping can easily get messy. The less structured the data that we're trying to gather is, the more tricky it is to parse it. In practice, web scraping works best for 
* Structured pages. The page we've just scraped is a text article with some header structure. Scraping data from tables or other more structured forms is often much easier. 
* Static pages. Accessing dynamic content (eg JavaScript) is often impossible through the standard HTML approach we have used. Alternative ways exist, but require more work.

Compared to downloading a file or using an API, web scraping is a fragile way to access a site. The HTML page could be changed, breaking your scraper, whereas APIs are often well maintained and robust. 

You'll have noticed that it would be very easy to crawl through a lot of pages on a site very quickly by looping requests. This could be on a news website, or for example looking for business analytics positions on an online jobs platform. Before you do this, note that many websites don't like crawlers: The Guardian's business model, for instance, depends on ad revenues, so they don't like their content being automatically accessed. Here we've downloaded just one page, but if you ran a big scraping operation on them, they might ban you (and your flatmates). They do have an [API](http://open-platform.theguardian.com/access/) for accessing content.

In brief, scraping can be a powerful tool, but if there's an alternative way to get the data you need, don't scrape manually. If you do scrape data, be nice to the website and follow their terms of access.

Here are some further resources on scraping:
* [Automate the Boring Stuff has a detailed chapter on scraping](https://automatetheboringstuff.com/chapter11/)
* [The Beautiful Soup documentation](http://www.crummy.com/software/BeautifulSoup/bs4/doc/) is very well written
* [A success story with scraping](https://www.wired.com/2014/01/how-to-hack-okcupid/all/)
* [A nice and well-documented application on the NY Times website](http://www.dataschool.io/python-web-scraping-of-president-trumps-lies/)
* [Web data collection tasks from Stanford's Computational Journalism lab](https://github.com/stanfordjournalism/search-script-scrape), mostly through various US government APIs

### Scraping challenge

Download the top 100 most popular books from Project Gutenberg as text files from [here](https://www.gutenberg.org/browse/scores/top). Hint: you'll need inspect the page HTML to figure out the right tags to find, and how to get to the text file following these links. Bonus: save the the files and analyse their content; for example, which of them has the longest average word length?
